# 🚀 Deploy a Trained Policy

This notebook guides you through deploying a trained policy to a physical robot. 

### Process:
1.  **Configure**: Set the path to your trained model and the robot's server address.
2.  **Load**: The `policy_loader` automatically loads the model and its training configuration.
3.  **Deploy**: The `deploy_policy` function starts the inference loop and sends commands to the robot.

The deployment script automatically handles details like the action space (`tcp`, `joint`, etc.) based on the loaded training configuration.

## 1. Configuration

First, specify the necessary parameters for deployment. **You must edit these values.**

In [1]:
import pathlib

# TODO: Change to the directory containing your trained policy checkpoint.
# Example: "outputs/2025-09-14/12-00-00"
CHECKPOINT_DIR = pathlib.Path("/home/jovyan/outputs/ditflow_image_sort_red_blocks_80/checkpoints/015000/pretrained_model")

# TODO: Change to the robot's IP address.
SERVER_ENDPOINT = "192.168.0.203:50051"

# Inference frequency in Hz. Higher values result in smoother but potentially faster movements.
INFERENCE_FREQUENCY_HZ: float = 5.0

print(f"Attempting to load policy from: {CHECKPOINT_DIR}")
print(f"Robot server endpoint: {SERVER_ENDPOINT}")
print(f"Inference frequency: {INFERENCE_FREQUENCY_HZ} Hz")

Attempting to load policy from: /home/jovyan/outputs/ditflow_image_sort_red_blocks_80/checkpoints/015000/pretrained_model
Robot server endpoint: 192.168.0.203:50051
Inference frequency: 5.0 Hz


## 2. Load the Policy

Now, we load the policy from the specified checkpoint directory. The loader will find the latest checkpoint and its corresponding configuration file.

In [2]:
from example_policies.robot_deploy import policy_loader

policy, cfg = policy_loader.load_policy(CHECKPOINT_DIR)

print("✅ Policy loaded successfully!")

Number of flow params: 42.11M
Loading weights from local directory
✅ Policy loaded successfully!


## 3. (Optional) Modify Policy Attributes

Before deployment, you can override policy attributes for experimentation. For example, you might want to adjust the action chunking (`n_action_steps`) to see how it affects robot behavior.

In [3]:
# Uncomment and modify the lines below to change policy attributes.
# For available options, refer to the lerobot policy's config documentation.

# Change the device on the config, not the policy!!
cfg.device = "cuda"
policy.to(cfg.device)  # or "cpu"
# policy.n_action_steps = 15  # Number of actions to predict in each forward pass

# print(f"Action steps set to: {policy.n_action_steps}")

DiTFlowImagePolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_state): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 18 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 18 (cuda:0)]
    )
    (buffer_observation_images_rgb_static): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
    )
    (buffer_observation_images_rgb_left): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
    )
    (buffer_observation_images_rgb_right): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
    )
  )
  (normalize_targets)

## 4. Deploy to Robot

Finally, execute the cell below to start sending commands to the robot.

⚠️ **Warning**: This will move the physical robot. Ensure the robot has a clear and safe workspace.

In [4]:
from example_policies.robot_deploy.deploy import deploy_policy, RobotClient

# CART_WAYPOINT is most stable and responsive. For legacy behaviour, use CART_QUEUE.
# JOINT_DIRECT and CART_DIRECT are less stable and not recommended at the moment
deploy_policy(
    policy,
    cfg, 
    hz=INFERENCE_FREQUENCY_HZ,
    server=SERVER_ENDPOINT,
    controller=RobotClient.CART_WAYPOINT,
)

Press Enter to start 


Starting inference loop...
{'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(18,)), 'observation.images.rgb_static': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 256, 256)), 'observation.images.rgb_left': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 256, 256)), 'observation.images.rgb_right': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 256, 256))}

=== RAW MODEL PREDICTION ===
=== Step 0 | Absolute TCP ===
 Left  Pos   | Obs [-0.340, 0.798, 0.413]  Pred [-0.349, 0.803, 0.427]
 Left  Quat  | Obs [0.364, -0.931, -0.002, 0.002]  Pred [-0.375, 0.928, 0.016, 0.003]
 Right Pos   | Obs [0.273, 0.801, 0.413]  Pred [0.234, 0.788, 0.339]
 Right Quat  | Obs [-0.000, -1.000, 0.020, 0.000]  Pred [-0.094, -1.000, -0.038, 0.024]
 Grippers (L,R) [0.000, 0.000]


=== ABSOLUTE ROBOT COMMANDS ===
=== Step 0 | Absolute TCP ===
 Left  Pos   | Obs [-0.340, 0.798, 0.413]  Pred [-0.349, 0.803, 0.427]
 Left  Quat  | Obs [0.364, -0.931

KeyboardInterrupt: 